In [2]:
import os

from rasterio.plot import reshape_as_image
from rasterio.plot import show
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import cascaded_union

import numpy as np
import cv2

from patchify import patchify
from PIL import Image

import urllib

import xml.etree.ElementTree as ET
import shapely
import geopandas as gpd

from io import BytesIO
from zipfile import ZipFile

from imageio import imread

rs = 42

In [3]:
import torch

# Download tile data

In [4]:
url_metadata = "https://www.opengeodata.nrw.de/produkte/geobasis/lusat/dop/dop_jp2_f10/dop_meta.zip"
trgt_filename = 'dop_nw.csv'

response = urllib.request.urlopen(url_metadata)
zipfile = ZipFile(BytesIO(response.read()))

metadata = pd.read_csv(zipfile.open(trgt_filename), 
                       sep=';', 
                       skiprows=5) # skip first 5 rows with irrelevant metadata

metadata.head(10)

,Kachelname,Erfassungsmethode,Aktualitaet,Bildflugnummer,Kamera_Sensor,Bodenpixelgroesse,Spektralkanaele,Koordinatenreferenzsystem_Lage,Koordinatenreferenzsystem_Hoehe,Bezugsflaeche,...,Anzahl_Zeilen,Farbtiefe,Standardabweichung,Dateiformat,Hintergrund,Quelldatenqualitaet,Kompression,Komprimierung,Belaubungszustand,Bemerkungen
0,dop10rgbi_32_375_5666_1_nw_2021,0,2021-06-02,1358/21 Leverkusen Wuppertal,DMCIII-27569_DMCIII,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",3,keine
1,dop10rgbi_32_438_5765_1_nw_2022,0,2022-03-10,1377/22 Greven Ibbenbüren,UCEM3-431S91898X119229-f100_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
2,dop10rgbi_32_366_5723_1_nw_2020,0,2020-03-23,1333/20 Wesel Marl,UCEp-1-31011051_UCEp,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
3,dop10rgbi_32_344_5645_1_nw_2021,0,2021-03-02,1355/21 Düsseldorf Kerpen,UCEM3-431S71678X_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
4,dop10rgbi_32_407_5744_1_nw_2022,0,2022-03-03,1379/22 Warendorf,DMCIII-27532_DMCIII,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
5,dop10rgbi_32_397_5744_1_nw_2022,0,2022-02-27,1378/22 Bocholt Coesfeld,UCEp-1-31011051-f100_UCEp,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
6,dop10rgbi_32_313_5624_1_nw_2021,0,2021-03-07,1356/21 Aachen Kronenburg,UCEM3-1-82416042_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
7,dop10rgbi_32_335_5702_1_nw_2020,0,2020-03-24,1334/20 Duisburg Herne,UCEM3-431S51194X_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
8,dop10rgbi_32_388_5791_1_nw_2022,0,2022-02-23,1376/22 Ahaus Rheine,UCEM3-431S41091X314298-f100_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine
9,dop10rgbi_32_304_5671_1_nw_2021,0,2021-02-20,1354/21 Mönchengladbach- Würselen,UCEM3-431S72402X_UCE-M3,10,RGBI,25832,7837,bDOM,...,10000,8,20,JPEG2000,0,1,1,"GDAL_JP2ECW, 90",1,keine


# Retrieving Shapefiles

In [5]:
def get_shapefile(bbox2:tuple, crs='EPSG:25832') -> gpd.GeoDataFrame:
    
    base_url = "https://www.wfs.nrw.de/geobasis/wfs_nw_alkis_vereinfacht?SERVICE=WFS&VERSION=2.0.0&REQUEST=GetFeature&TYPENAMES=ave:GebaeudeBauwerk&BBOX="
    
    x, y = bbox2                                # unpack tuple
    x2 = x + 1000                               # get second lat/lon value for bounding box (always 10000*10000)
    y2 = y + 1000
    
    bbox4 = (x, y, x2, y2)
    
    bbox_str = ','.join(list(map(str, bbox4)))  # create bounding box string for API query
    
    gml_url = ''.join([base_url, bbox_str])
    
    req = urllib.request.Request(gml_url)       # query webservice
    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    
    gml_str = response.read()
    
    root = ET.ElementTree(ET.fromstring(gml_str)).getroot() # response is formatted as GML, which can be queried like normal XML, by referencing the relevant namespaces
    
    
    namespace = {'gml': "http://www.opengis.net/gml/3.2",
             'xmlns': "http://repository.gdi-de.org/schemas/adv/produkt/alkis-vereinfacht/2.0",
             'wfs': "http://www.opengis.net/wfs/2.0",
             'xsi': "http://www.w3.org/2001/XMLSchema-instance"
             }
    
    buildings = [i.text for i in root.findall('.//gml:posList', namespace)]
    
    funktions = [i.text for i in root.iter('{http://repository.gdi-de.org/schemas/adv/produkt/alkis-vereinfacht/2.0}funktion')]
    
    ids = [i.items()[0][1] for i in root.findall('.//gml:MultiSurface[@gml:id]', namespace)]
    
    building_shapefiles = []
    
    for id, funktion, build in zip(ids, funktions, buildings):
        coord_iter = iter(build.split(' '))                                                 # coordinates are not in the correct format, therefore need to be rearranged            
        coords = list(map(tuple, zip(coord_iter, coord_iter)))
        
        poly = shapely.geometry.Polygon([[float(p[0]), float(p[1])] for p in coords])       # create shapefile from points
        
        building_shapefiles.append({'id': id, 'funktion':funktion, 'geometry': poly})       # create records of each building on the selected tile
    
    df = pd.DataFrame.from_records(building_shapefiles)
    gdf = gpd.GeoDataFrame(df, crs=crs)                                                     # return geopandas dataframe for input that can be passed to the mask generation 
    
    return gdf

# Combine shapefile to polygon and generate mask

In [6]:
def poly_from_utm(polygon, transform):
    poly_pts = []
    
    poly = shapely.ops.unary_union(polygon)
    for i in np.array(poly.exterior.coords):
        
        # Convert polygons to the image CRS
        poly_pts.append(~transform * tuple(i))
        
    # Generate a polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

def generate_mask(shapefiles, img_url):
    
    with rasterio.open(img_url, "r") as src:
        raster_img = src.read()
        raster_meta = src.meta
    
    # Generate binary mask
    polygons = []
    im_size = (raster_meta["height"], raster_meta["width"])
    for _, row in shapefiles.iterrows():
        if row['geometry'].geom_type == 'Polygon':
            poly = poly_from_utm(row['geometry'], raster_meta["transform"])
            polygons.append(poly)
        else:
            for p in row['geometry']:
                poly = poly_from_utm(p, raster_meta["transform"])
                polygons.append(poly)

    mask = rasterize(shapes=polygons, out_shape=im_size)
    
    return mask, raster_img

# Patchify and save images and masks

In [7]:
def load_and_patchify(img, patch_size, output_path, tile_identifier, num_channels=None):
    
    if num_channels:                                                            # this handles pictures
        size_x = (img.shape[1]//patch_size) * patch_size
        size_y = (img.shape[2]//patch_size) * patch_size
        
        img = img[:, :size_x, :size_y]                                          # subsets image (input size is not neccessarily divisible by patch size)
        
        patch_img = patchify(img, (num_channels, patch_size, patch_size), step=patch_size)
        patch_img = np.squeeze(patch_img)
        
    else:                                                                       # this handles masks
        size_x = (img.shape[0]//patch_size) * patch_size
        size_y = (img.shape[1]//patch_size) * patch_size
    
        img = img[:size_x, :size_y] * 255                                       # mask needs to be multiplied by 255, as it is on a 0-1 scale
        
        patch_img = patchify(img, (patch_size, patch_size), step=patch_size) 
    
    for i in range(patch_img.shape[0]):                                             # this could also be left out, we could just return numpy arrays and pass them to the model.
        for k in range(patch_img.shape[1]):
            single_patch_img = patch_img[i, k]                                  # iterates through all patches
            
            path_string = str(tile_identifier) + '_' + str(i) + '_' + str(k) + '.png'
            
            file_path = os.path.join(output_path, path_string)
                
            if num_channels:
                single_patch_img = single_patch_img.swapaxes(0,2)
            
            os.makedirs(os.path.dirname(file_path), exist_ok = True)
            #print(file_path)
            #print(single_patch_img)
            #break
            cv2.imwrite(file_path, single_patch_img)                            # writes the image to this pass

# Complete pipeline

1. *get_shapefile*:
- **Input**: bounding box values (only north and east, rest is inferred from tile size) as a tuple
- **Output**: geopandas dataframe with polygons of all buildings on the tile

2. *generate_mask*:
- **Input**: geopandas dataframe and tile-image path
- **Output**: mask and image in 1000-1000 pixels

3. *load_and_patchify*:
- **Input**: mask OR image, patch_size (**should correspond to input size for model**), path to output folder (e.g. masks or images), a string identifying each individual 1000-1000 tile (needs to be unique, otherwise output will be overwritten), number of channels (for masks: None, for images: 4)
- **Output**: saves individual images as png files into the specified output folder

# Example

In [42]:
# Download metadata (cell 2)

# index data from metadata dataframe to get coordinates and image link
random_index = np.random.choice(metadata.index.values, 1)

lat = metadata.loc[random_index[0], 'Koordinatenursprung_East']
long = metadata.loc[random_index[0], 'Koordinatenursprung_North']
coords = (lat, long)


base_url = "https://www.opengeodata.nrw.de/produkte/geobasis/lusat/dop/dop_jp2_f10/"
img_path = metadata.loc[random_index[0], 'Kachelname']

# create image url from base url, image url and file extension
img_url = base_url + img_path + '.jp2'

In [43]:
shp_data = get_shapefile(coords)

mask, image = generate_mask(shp_data, img_url)

patch_size = 256

base_path_krishna = r"C:\Users\krish\building-segmentation-tutorial"

#load_and_patchify(image, patch_size, '../output/patchified_imgs/', random_index[0], 4)
load_and_patchify(image, patch_size, base_path_krishna + '\\output\\patchified_imgs\\', random_index[0], 4)

#load_and_patchify(mask, patch_size, '../output/patchified_masks/', random_index[0])
load_and_patchify(mask, patch_size, base_path_krishna + '\\output\\patchified_masks\\', random_index[0])

C:\Users\krish\building-segmentation-tutorial\output\patchified_imgs\21310_0_0.png
[[[ 51  64  57 179]
  [ 54  76  64 187]
  [ 58  89  72 195]
  ...
  [ 74  88  74 199]
  [ 75  89  74 200]
  [ 76  89  74 200]]

 [[ 52  74  68 188]
  [ 51  77  66 187]
  [ 51  80  64 186]
  ...
  [ 73  87  72 198]
  [ 75  89  73 199]
  [ 76  90  74 200]]

 [[ 53  84  79 198]
  [ 49  77  67 187]
  [ 45  71  56 177]
  ...
  [ 73  87  72 197]
  [ 75  89  73 199]
  [ 77  91  74 201]]

 ...

 [[ 96 113  86 215]
  [ 95 112  86 214]
  [ 95 112  87 214]
  ...
  [ 36  56  62 116]
  [ 36  56  62 115]
  [ 36  56  62 114]]

 [[ 96 113  87 215]
  [ 95 112  87 214]
  [ 95 112  87 214]
  ...
  [ 36  56  62 117]
  [ 36  56  62 116]
  [ 36  56  62 115]]

 [[ 96 113  88 215]
  [ 95 112  87 214]
  [ 95 112  87 214]
  ...
  [ 36  56  61 117]
  [ 36  56  62 116]
  [ 36  56  62 115]]]
C:\Users\krish\building-segmentation-tutorial\output\patchified_imgs\21310_1_0.png
[[[ 78  90  74 201]
  [ 78  91  74 200]
  [ 79  92  74 200]


In [45]:
nj = np.array(image)

array([[[ 51,  52,  53, ...,  35,  35,  35],
        [ 54,  51,  49, ...,  34,  34,  34],
        [ 58,  51,  45, ...,  34,  34,  34],
        ...,
        [ 59,  56,  54, ...,  62,  57,  57],
        [ 59,  56,  54, ...,  63,  61,  61],
        [ 59,  56,  54, ...,  63,  61,  61]],

       [[ 64,  74,  84, ...,  56,  56,  56],
        [ 76,  77,  77, ...,  55,  55,  55],
        [ 89,  80,  71, ...,  55,  55,  55],
        ...,
        [ 75,  72,  70, ...,  87,  82,  82],
        [ 75,  72,  70, ...,  83,  81,  81],
        [ 75,  72,  70, ...,  83,  81,  81]],

       [[ 57,  68,  79, ...,  63,  63,  63],
        [ 64,  66,  67, ...,  62,  62,  62],
        [ 72,  64,  56, ...,  61,  61,  61],
        ...,
        [ 69,  67,  65, ...,  75,  70,  70],
        [ 69,  67,  65, ...,  71,  69,  69],
        [ 69,  67,  65, ...,  71,  69,  69]],

       [[179, 188, 198, ..., 129, 129, 129],
        [187, 187, 187, ..., 128, 128, 128],
        [195, 186, 177, ..., 128, 128, 128],
        ..

A function to save the generated patches as a tensor

In [8]:
def load_and_patchify_tensor(img, patch_size, tile_identifier, num_channels=None):
    
    if num_channels:
        size_x = (img.shape[1]//patch_size) * patch_size
        size_y = (img.shape[2]//patch_size) * patch_size
        
        img = img[:, :size_x, :size_y]
        
        patch_img = patchify(img, (num_channels, patch_size, patch_size), step=patch_size)
        patch_img = np.squeeze(patch_img)
        
    else:
        size_x = (img.shape[0]//patch_size) * patch_size
        size_y = (img.shape[1]//patch_size) * patch_size
    
        img = img[:size_x, :size_y] * 255
        
        patch_img = patchify(img, (patch_size, patch_size), step=patch_size) 
    
    for i in range(patch_img.shape[0]):
        for k in range(patch_img.shape[1]):
            single_patch_img = patch_img[i, k]
            if num_channels:
                single_patch_img = single_patch_img.swapaxes(0,2)
            
            yield torch.Tensor(single_patch_img)

Generates patches for tiles in bulk and saves them as a single tensor

In [27]:
Y = [(0,0)]
Msks = []
Imgs = []
for y in range(20):    
    
    
    coords = (0,0)
    while coords in Y:
        random_index = np.random.choice(metadata.index.values, 1)
        lat = metadata.loc[random_index[0], 'Koordinatenursprung_East']
        long = metadata.loc[random_index[0], 'Koordinatenursprung_North']
        coords = (lat, long)

    try:
        base_url = "https://www.opengeodata.nrw.de/produkte/geobasis/lusat/dop/dop_jp2_f10/"
        img_path = metadata.loc[random_index[0], 'Kachelname']

        # create image url from base url, image url and file extension
        img_url = base_url + img_path + '.jp2'

        shp_data = get_shapefile(coords)

        mask, image = generate_mask(shp_data, img_url)

        patch_size = 256

        base_path_krishna = r"C:\Users\krish\building-segmentation-tutorial"


        imgs = [i for i in load_and_patchify_tensor(image, patch_size, random_index[0], 4)]
        msks = [i for i in load_and_patchify_tensor(mask, patch_size, random_index[0])]

        msk = 0
        while msk < len(msks):
            if torch.count_nonzero(msks[msk]).item() == 0:
                rem = np.random.choice(range(100))
                if rem > 20:
                    del msks[msk]
                    del imgs[msk]
                    continue
            msk += 1
            
        if y == 0:
            Msks = torch.stack(msks)
            Imgs = torch.stack(imgs)
        else:
            Msks = torch.cat((Msks, torch.stack(msks)), 0)
            Imgs = torch.cat((Imgs, torch.stack(imgs)), 0)
        #Msks.extend(msks)
        #Imgs.extend(imgs)
        print(coords)
        print(str(y) + "/100")
    except:
        continue

                
    


(341000, 5721000)
0/100
(438000, 5658000)
1/100
(400000, 5674000)
2/100
(326000, 5616000)
4/100
(359000, 5778000)
5/100
(379000, 5702000)
7/100
(462000, 5663000)
9/100
(482000, 5780000)
10/100
(388000, 5689000)
11/100
(325000, 5695000)
12/100
(386000, 5700000)
13/100
(397000, 5734000)
14/100
(340000, 5675000)
15/100
(514000, 5707000)
16/100
(403000, 5744000)
17/100
(355000, 5718000)
18/100
(470000, 5731000)
19/100


In [31]:
torch.save(Imgs, r"C:\Users\krish\building-segmentation-tutorial\output\tensors\Imgs.pt")
torch.save(Msks, r"C:\Users\krish\building-segmentation-tutorial\output\tensors\Msks.pt")